# Verification torch

查看PyTorch版本，cuda版本等等

In [ ]:
import torch
import numpy as np

print(torch.__version__)  # 查看 PyTorch 版本
print(torch.version.cuda) # 查看 PyTorch 构建时使用的 CUDA 版本
print(torch.cuda.is_available()) # 最重要：输出应为 True

# Tensors

高维数据结构，任意维的矩阵

## Initializing a Tensor

In [ ]:
# From list
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data)
# From numpy
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)
# From tensor
# *_like based a existing tensor's template, create tensors with new content but have same structure.
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")
x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

## `shape` is a tuple of tensor dimensions.

In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

## Attributes of tensor

Tensor attributes describe their shape, datatype, and the device on which they are stored.

In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

## Operations on Tensors

Over 1200 tensor operations, including arithmetic, linear algebra, matrix manipulation (transposing, indexing, slicing), sampling and more are comprehensively described [here](https://pytorch.org/docs/stable/torch.html).

### reshape operations

Allocate an accelerator by going to Runtime > Change runtime type > GPU.

Use `.to` to > GPU

In [ ]:
tensor = tensor.to(torch.accelerator.current_accelerator())
print(f"Device tensor is stored on: {tensor.device}")

Use `torch.cat` or `torch.stack` to concatenate a sequence of tensors along a given dimension.

In [ ]:
tensor = torch.ones(4, 4)
t1 = torch.cat([tensor, tensor, tensor], dim=1) # dim 希望拼接在哪个维度上
print(t1)
t2 = torch.stack([tensor, tensor, tensor], dim=1)
print(t2)

### Arithmetic operations

In [ ]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor 转置
y1 = tensor @ tensor.T # @ is alias of matmul
y2 = tensor.matmul(tensor.T)
# y1 is equal to y2
y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)
print(y3)
# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)
print(z3)

Use `.item()` to get the value of single-element tensor 

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

A function with `x.*_` will consume `x`

In [ ]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)
x = tensor.add(5)
print(x)

## Bridge with numpy

A change in the tensor reflects in the NumPy array.

In [ ]:
# tensor to numpy
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")
t.add_(5)
print(f"t: {t}")
print(f"n: {n}")

In [ ]:
# numpy to tensor
n = np.ones(5)
t = torch.from_numpy(n)
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# DataSets & DataLoaders

`torch.uilt.data.DataLoader` and `torch.uilt.data.Dataset` are the two most commonly used utilities for loading data in PyTorch.

`Dataset` store the samples and corresponding labels, and `DataLoader` wraps an iterable around the `Dataset` to enable easy access to the samples.

## Iterating and Visualizing the Dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Create a Custom Dataset for your files

Create a class from Dataset calss and override the following methods:
1. __init__ 
2. __len__ return the number of samples in your dataset
3. __getitem__ get a tensor and a label of a sample via its index

In [ ]:
import os
import pandas as pd
from torchvision.io import decode_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = decode_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

## DataLoader

When loading data from the dataset, you can specify the size of each batch of data and whether it is shuffled.

It is a iterator.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

# Transforms

Let the data more suitable for training.

`transform` to modify the input data (feature).

`target_transform` to modify the target data (label).

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get device for training.

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define a Neural Network by subclassing `nn.Module`

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # 2 dim to 1 dim
        self.linear_relu_stack = nn.Sequential( # leayers
            nn.Linear(28*28, 512),  # 全连接层 28*28 -> 512
            nn.ReLU(),              # 修正线性单元
            nn.Linear(512, 512),    # 全连接层 512 -> 512
            nn.ReLU(),              # 修正线性单元
            nn.Linear(512, 10),     # 全连接层 512 -> 10
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Move neural network to device

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device) # Suppose this is the input
logits = model(X) # [ list of socre[], funciton of ]
pred_probab = nn.Softmax(dim=1)(logits)
print(f"Predicted class: {pred_probab}")
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0836, -0.0828, -0.0099,  0.0137,  0.0428,  0.0856,  0.0937, -0.0362,
          0.0164,  0.0518]], grad_fn=<AddmmBackward0>)
Predicted class: tensor([[0.0910, 0.0911, 0.0979, 0.1003, 0.1032, 0.1078, 0.1086, 0.0954, 0.1006,
         0.1042]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([6])


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")